<a href="https://colab.research.google.com/github/prateekanand23/Arthtattva-Case-Assignment/blob/main/Arthattva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from flask import Flask, request, jsonify
!pip install rapidfuzz
from rapidfuzz import process, fuzz
import pandas as pd

app = Flask(__name__)

# Load or initialize the mapping dictionary
try:
    mappings = pd.read_csv("product_mappings.csv").set_index("raw_name").to_dict()["standard_name"]
except FileNotFoundError:
    mappings = {}

def normalize_name(name):
    return ' '.join(name.lower().strip().split())

@app.route('/match', methods=['POST'])
def match_product():
    product_name = request.json.get('product_name')
    normalized_name = normalize_name(product_name)

    # Check for exact match
    if normalized_name in mappings:
        return jsonify({"status": "matched", "standard_name": mappings[normalized_name]})

    # Find close matches
    matches = process.extract(normalized_name, mappings.keys(), scorer=fuzz.partial_ratio, limit=3)
    suggestions = [match[0] for match in matches if match[1] > 80]  # Confidence threshold

    if suggestions:
        return jsonify({"status": "suggest", "suggestions": suggestions})
    else:
        return jsonify({"status": "not_matched", "message": "No close matches found. Please provide a manual mapping."})

@app.route('/add_mapping', methods=['POST'])
def add_mapping():
    raw_name = normalize_name(request.json.get('raw_name'))
    standard_name = request.json.get('standard_name')

    mappings[raw_name] = standard_name
    pd.DataFrame(mappings.items(), columns=["raw_name", "standard_name"]).to_csv("product_mappings.csv", index=False)
    return jsonify({"status": "success", "message": f"Mapping for '{raw_name}' added."})


if __name__ == "__main__":
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
